In [1]:
import pandas as pd
import numpy as np
import preprocessing as pre
import time

## 3 tenure: 1997 - 2001

In [2]:
k = 3
xml_file_path = '.\Sejm-20240316T162344Z-001\Sejm\GLO3k1p119p_i\GLO3k1p119p_i.xml'
csv_file = 'GLO3k1p119p_i.csv'

In [3]:
t1 = time.time()
df_all = pre.save_xml_as_csv(xml_file_path, csv_file)
t2 = time.time()

In [4]:
print(f'{(t2 - t1)/60:.2f} min')

1.67 min


In [5]:
df_all.head()

,vote_id,data_godzina,Nrleg,NazwiskoImie,Glos,Klub
0,3001001,1997-10-20 12:51:54,1,Adamczyk Franciszek,Za,AWS
1,3001001,1997-10-20 12:51:54,2,Adamska-Wedler Elżbieta,Za,AWS
2,3001001,1997-10-20 12:51:54,3,Adamski Władysław,Za,SLD
3,3001001,1997-10-20 12:51:54,4,Ajchler Romuald,Za,SLD
4,3001001,1997-10-20 12:51:54,5,Anusz Andrzej,Za,AWS


In [6]:
assert np.all(df_all.dtypes.values == [np.dtype('int32'), np.dtype('<M8[ns]'), np.dtype('int32'), np.dtype('O'), np.dtype('O'), np.dtype('O')])
assert df_all.isnull().sum().sum() == 0
assert df_all[df_all.duplicated()].size == 0

In [7]:
df_all['data_godzina'].min(), df_all['data_godzina'].max()

(Timestamp('1997-10-20 12:51:54'), Timestamp('2001-09-18 16:26:05'))

In [8]:
y_m_list = pd.date_range(df_all['data_godzina'].min(), df_all['data_godzina'].max(), 
              freq='M').strftime("%Y-%m").tolist()

y_m_list.append(df_all['data_godzina'].max().strftime("%Y-%m"))

In [9]:
y_m_list[0], y_m_list[-1]

('1997-10', '2001-09')

In [10]:
y_m_list = [x.split('-') for x in y_m_list]

In [11]:
t1 = time.time()
df_deputies = pre.assign_node_ids(df_all)
t2 = time.time()

In [12]:
print(f'{(t2 - t1)/60:.2f} min')

1.07 min


### Monthly grahps

In [13]:
for year, month in y_m_list:
    df_m = df_all[(df_all['data_godzina'].dt.month == int(month)) & (df_all['data_godzina'].dt.year == int(year))]
    
    if len(df_m) == 0:
        print(year, month, 'no votings')
        continue
    
    df_m = pd.merge(df_m[['vote_id', 'data_godzina', 'Nrleg', 'NazwiskoImie', 'Glos', 'Klub']], df_deputies[['Nrleg', 'NazwiskoImie', 'node_id']], on=['Nrleg', 'NazwiskoImie'], how='left')
    df_node_atr = pre.assign_attributes(df_m)
    
    edge_matrix = pre.egde_weights(df_m)
    
    pre.create_graph(edge_matrix, df_node_atr, file_name=f"graphs/G_{k}_{year}_{month}")

100%|████████████████████████████████████████████████████████████████████████████████| 159/159 [00:12<00:00, 13.08it/s]


1999 08 no votings


100%|████████████████████████████████████████████████████████████████████████████████| 301/301 [00:29<00:00, 10.06it/s]


2000 08 no votings


100%|████████████████████████████████████████████████████████████████████████████████| 248/248 [00:19<00:00, 12.47it/s]


In [14]:
t3 = time.time()

In [15]:
print(f'{(t3 - t2)/60:.2f} min')

23.53 min


### 4 years graph - whole tenure

In [ ]:
df_all = pd.merge(df_all, df_deputies[['Nrleg', 'NazwiskoImie', 'node_id']], on=['Nrleg', 'NazwiskoImie'], how='left')
df_node_atr = pre.assign_attributes(df_all)

edge_matrix = pre.egde_weights(df_all)

pre.create_graph(edge_matrix, df_node_atr, file_name=f"graphs/G_{k}")
t4 = time.time()

 25%|██████████████████▍                                                      | 2988/11801 [1:15:05<3:41:30,  1.51s/it]

In [ ]:
print(f'{(t4 - t3)/60:.2f} min')

In [ ]:
# G = pickle.load(open('filename.pickle', 'rb'))